In [1]:
import nltk
import numpy as np
import tensorflow as tf
from datasets import load_dataset, load_metric
from tensorflow.keras.callbacks import TensorBoard
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamWeightDecay, TFT5ForConditionalGeneration, T5Tokenizer

ModuleNotFoundError: No module named 'tensorflow.keras'

## <center>SOURCE:</center>
<center>https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization-tf.ipynb</center>

In [2]:
# Get dataset and remove column "id"
dataset = load_dataset("cnn_dailymail")
metric = load_metric("rouge")

C:\Users\Yoga\AppData\Local\Temp\ipykernel_12740\3245786012.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
D:\anaconda3\lib\site-packages\datasets\load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 11490
    })
})

In [4]:
model_checkpoint =  "google-t5/t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Show some examples picked randomly in the dataset.

In [5]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

If we are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [7]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

Write the function that will preprocess our samples. feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [8]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["highlights"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
preprocess_function(dataset["train"][:2])

D:\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[938, 3, 5, 3, 26138, 3373, 3, 5, 3, 10744, 8775, 20619, 2326, 10, 3, 5, 968, 10, 2596, 3, 6038, 6, 944, 1797, 2038, 3, 5, 1820, 3, 5, 3, 6880, 4296, 11430, 10, 3, 5, 627, 10, 3420, 3, 6038, 6, 944, 1797, 2038, 3, 5, 37, 25214, 13, 8, 5186, 839, 6502, 309, 25985, 16, 1117, 16711, 65, 6666, 6149, 3986, 13, 2078, 724, 16, 5186, 839, 6, 2698, 242, 157, 7, 11, 2549, 3540, 12, 8, 3, 88, 7768, 17, 159, 71, 6722, 16, 1480, 1600, 11, 778, 1797, 5, 37, 538, 1685, 1775, 65, 4683, 46, 18599, 13, 4773, 21, 1321, 113, 5526, 874, 9894, 11, 808, 30124, 5, 16098, 1079, 17796, 9, 41, 22665, 61, 13, 8, 5186, 839, 6502, 309, 25985, 16, 1117, 16711, 65, 6666, 6149, 3986, 13, 2078, 724, 16, 5186, 839, 6, 2698, 242, 157, 7, 11, 2549, 3540, 12, 8, 3, 88, 7768, 17, 159, 71, 3, 5, 1015, 18502, 1707, 2350, 3440, 11019, 120, 571, 3820, 845, 8, 1020, 19, 731, 6, 68, 4298, 473, 34, 31, 7, 359, 12, 5685, 151, 12, 8, 487, 4773, 5, 37, 3, 26, 25985, 2162, 30, 2089, 24, 16098, 1079, 17796, 9, 19, 838, 9

To apply this function on all the pairs of sentences in the dataset,  use the `map` method of the `dataset` object  created earlier. This will apply the function on all the elements of all the splits in `dataset`, so the training, validation and testing data will be preprocessed in one single command.

In [10]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map: 100%|███████████████████████████████████████████████████████████████| 11490/11490 [00:25<00:00, 449.45 examples/s]


Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is sequence-to-sequence (both the input and output are text sequences), we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [15]:
model = TFT5ForConditionalGeneration.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


We set some parameters like the `learning rate` and the `batch_siz` eand customize the `weight decay`.

In [16]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

We need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels. Note that our data collators are designed to work for multiple frameworks, so ensure you set the `return_tensors='np'` argument to get NumPy arrays out, our TF dataset pipeline actually uses a NumPy loader internally, which is wrapped at the end with a `tf.data.Dataset`. As a result, `np` is usually more reliable and performant when you're using it!

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, pad_to_multiple_of=128)

In [18]:
tokenized_datasets["train"]

Dataset({
    features: ['id', 'article', 'highlights', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 287113
})

Next, we convert our datasets to `tf.data.Dataset`, which Keras understands natively. Using [`Model.prepare_tf_dataset()`](https://huggingface.co/docs/transformers/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset). The `Model` method can inspect the model to determine which column names it can use as input, which means we don't need to specify them yourself.

In [24]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [39]:
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

The callback takes two main arguments - a `metric_fn` and an `eval_dataset`. It then iterates over the `eval_dataset` and collects the model's outputs for each sample, before passing the `list` of predictions and the associated `list` of labels to the user-defined `metric_fn`. If the `predict_with_generate` argument is `True`, then it will call `model.generate()` for each input sample instead of `model.predict()` - this is useful for metrics that expect generated text from the model, like `ROUGE`.

This callback allows complex metrics to be computed each epoch that would not function as a standard Keras Metric. Metric values are printed each epoch, and can be used by other callbacks like `TensorBoard` or `EarlyStopping`.

In [ ]:
tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./summarization_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
)

callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

In [47]:
model.fit(
    train_dataset, 
    validation_data=validation_dataset, 
    epochs=1, 
)

AttributeError: in user code:

    File "D:\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "D:\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "D:\anaconda3\lib\site-packages\transformers\modeling_tf_utils.py", line 1571, in train_step
        x, y, sample_weight = keras.utils.unpack_x_y_sample_weight(data)

    AttributeError: module 'keras.utils' has no attribute 'unpack_x_y_sample_weight'


## <center>SOURCE:</center>
<center>https://huggingface.co/docs/transformers/tasks/summarization</center>

In [ ]:
# Get dataset and remove column "id"
dataset = load_dataset("cnn_dailymail")

In [ ]:
dataset

In [ ]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf"

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
model.fit(x=tf_train_set, 
          validation_data=tf_test_set, 
          epochs=3)